In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.preprocessing import OneHotEncoder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Menampilkan struktur tabel (hht, bps, dan bulan)
dataset = pd.read_csv("/content/drive/MyDrive/.../BPS.csv")[12:144].replace("-", 0).drop(columns=["Unnamed: 0"])
trendhht = pd.read_csv("/content/drive/MyDrive/.../compare.txt")["hht"][:132].astype("int64")
dataset["BULAN"] = dataset.BULAN.str.replace("-", "")
dataset["BULAN"] = dataset["BULAN"].str[:6]
dataset = dataset.reset_index().astype("int64")
print(dataset, trendhht)

     index   BULAN      BPS
0       12  200901   473165
1       13  200902   421555
2       14  200903   511314
3       15  200904   487121
4       16  200905   521735
..     ...     ...      ...
127    139  201908  1530268
128    140  201909  1388719
129    141  201910  1346434
130    142  201911  1280781
131    143  201912  1377067

[132 rows x 3 columns] 0       417831
1       418780
2       420960
3       424360
4       428912
        ...   
127    1422431
128    1424262
129    1425873
130    1427274
131    1428477
Name: hht, Length: 132, dtype: int64


In [ ]:
# Memanggil data Tahun 2019
sample = np.array(dataset.BULAN)
sample_train = sample[:120].reshape(1, len(dataset.BULAN[:120]))
value_train = np.array(trendhht[:120]).astype("int64").reshape(1, len(trendhht[:120]))
sample_test = sample[120:].reshape(1, len(dataset.BULAN[120:]))
value_test = np.array(trendhht[120:]).astype("int64").reshape(1, len(trendhht[120:]))
print(sample_test)
print(value_test)

[[201901 201902 201903 201904 201905 201906 201907 201908 201909 201910
  201911 201912]]
[[1402521 1406188 1409571 1412674 1415500 1418061 1420368 1422431 1424262
  1425873 1427274 1428477]]


In [ ]:
# Membangun arsitektur LSTM Forecasting
lstm = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=202113, output_dim=50, trainable=False),
    tf.keras.layers.LSTM(100, time_major=True, return_sequences=True, activation="exponential", dropout = 0.2),
    tf.keras.layers.Dense(1, activation="exponential")
], name="LSTM")

lstm.compile(optimizer="RMSprop", loss="mean_absolute_percentage_error", metrics=["mean_absolute_percentage_error"])
lstm.summary()

Model: "LSTM"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 50)          10105650  
                                                                 
 lstm (LSTM)                 (None, None, 100)         60400     
                                                                 
 dense (Dense)               (None, None, 1)           101       
                                                                 
Total params: 10,166,151
Trainable params: 60,501
Non-trainable params: 10,105,650
_________________________________________________________________


In [ ]:
#Proses pelatihan LSTM Forecasting untuk data latih sekaligus melakukan validasi terhadap data uji
lstm.fit(sample_train, value_train, batch_size=1, epochs=5000, verbose=1, shuffle=False)

Output streaming akan dipotong hingga 5000 baris terakhir.
1/1 [==============================] - 0s 13ms/step - loss: 23.2328 - mean_absolute_percentage_error: 23.2328
Epoch 2502/5000
1/1 [==============================] - 0s 9ms/step - loss: 24.3555 - mean_absolute_percentage_error: 24.3555
Epoch 2503/5000
1/1 [==============================] - 0s 10ms/step - loss: 21.8079 - mean_absolute_percentage_error: 21.8079
Epoch 2504/5000
1/1 [==============================] - 0s 10ms/step - loss: 23.3950 - mean_absolute_percentage_error: 23.3950
Epoch 2505/5000
1/1 [==============================] - 0s 13ms/step - loss: 22.2684 - mean_absolute_percentage_error: 22.2684
Epoch 2506/5000
1/1 [==============================] - 0s 10ms/step - loss: 19.9183 - mean_absolute_percentage_error: 19.9183
Epoch 2507/5000
1/1 [==============================] - 0s 22ms/step - loss: 21.7677 - mean_absolute_percentage_error: 21.7677
Epoch 2508/5000
1/1 [==============================] - 0s 15ms/step - loss: 

In [ ]:
# Memprediksi tahun 2019 kemudian menampilkannya
test_predict = lstm.predict(sample_test, verbose=1)
test_predict.reshape(1, 12, 1)
predict = []
for i in test_predict:
    predict.append(i)

predict

1/1 [==============================] - 0s 394ms/step


[array([[688968.75],
        [863703.1 ],
        [677990.5 ],
        [874446.1 ],
        [739713.4 ],
        [587702.2 ],
        [889675.7 ],
        [733226.5 ],
        [618756.06],
        [635346.06],
        [929035.6 ],
        [937937.1 ]], dtype=float32)]

In [ ]:
# Hasil RMSE dan MAPE ditahun 2019
mape = []
rmse = []
for i in range(len(sample_test)):
    mape.append(abs((value_test.reshape(1, 12, 1)[i] - predict[i]) / value_test.reshape(1, 12, 1)[i]) * 100)
    rmse.append(np.sqrt((value_test.reshape(1, 12, 1)[i] - predict[i])**2))

# print("rmse", rmse)
# print("mape", mape)
mape = [j for sub in mape for j in sub]
rmse = [j for sub in rmse for j in sub]
eval = pd.DataFrame()
eval["MAPE"] = mape
eval["RMSE"] = rmse

eval

,MAPE,RMSE
0,[50.87640398967288],[713552.25]
1,[38.57840310115006],[542484.875]
2,[51.90093297889925],[731580.5]
3,[38.099934946066824],[538227.875]
4,[47.741902154715646],[675786.625]
5,[58.555930421892995],[830358.8125]
6,[37.363015253793385],[530692.3125]
7,[48.45257871910834],[689204.5]
8,[56.5560225225415],[805505.9375]
9,[55.44160928077044],[790526.9375]


In [ ]:
mapea=np.average(eval.MAPE)
rmsea = np.average(eval.RMSE)
print("MAPE: ", mapea)
print("RMSE: ", rmsea)

MAPE:  [46.06793205]
RMSE:  [653058.23958333]
